# Artificial Neural Networks

In this part you will learn:

- The Intuition of ANNs

- How to build an ANN

- How to predict the outcome of a single observation (Homework Challenge)

- How to evaluate the performance of an ANN with k-Fold Cross Validation

- How to tackle overfitting with Dropout

- How to do some Parameter Tuning on your ANN to improve its performance

- Enjoy Deep Learning!

### Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


# Part 1 - Data Preprocessing


### Import the dataset

In [2]:
dataset = pd.read_csv("https://raw.githubusercontent.com/Nath19/Deep_Learning/master/dataset/Churn_Modelling.csv")

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
dataset.shape

(10000, 14)

In [5]:
list(dataset.columns)

['RowNumber',
 'CustomerId',
 'Surname',
 'CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited']

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Declare dependant and independant variable

In [7]:
X = dataset.iloc[:,3:13]
y = dataset['Exited']

In [8]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


### Encoding categorical data

In [9]:
# Encoding categorical data
#We have two features like that : geography and sexe


labelencoder_X_1 = LabelEncoder()
X['Geography'] = labelencoder_X_1.fit_transform(X['Geography'])

labelencoder_X_2 = LabelEncoder()
X['Gender'] = labelencoder_X_2.fit_transform(X['Gender'])


### Splitting the dataset into the Training set and Test set


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [11]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2 - Now let's make the ANN!


In [12]:
import keras

Using TensorFlow backend.
/Users/nathanamar/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/nathanamar/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/nathanamar/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/nathanamar/anaconda3/lib/python3.7

In [25]:
from keras.models import Sequential
from keras.layers import Dense #for the layers
from keras.layers import Dropout

In [14]:
#initialize the ANN
classifier = Sequential()

In [15]:
# Adding the input layer and the first hidden layer
# Adding the input layer and the first hidden layer
#output_dim = 6 because there is 11 input + 1 output = 12/2
#kernel_initializer for the weights to have small number close to 0 with 'uniform'
#activation function we choosed 'relu' for the hidden mlayer and the sigmoid fir output layer
#input dim is the number of nodes as inputs
#With dropout : p =0.1 10% neurons gonna be disabled
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))
classifier.add(Dropout((p=0.1)))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout((p=0.1)))

# Adding the output layer

# Adding the output layer
#one node in the ouput so unit = 1
#sigmoid function  for the binary variable
#if we have more than a binary output to predict, for example three class : we have to change the units to 3 and choose the multiclass function that is softmax
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN

#optimizer is to find the optimal weights
#it's gonna be the gradien descent
#loss for the loss function within the gradient descent

#as we have a binary output the loss function is binary_crossentropy
#if we had multiclass output the loss is categorial_crrossentropy
#accuracy to evaluate our model
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 50)


Instructions for updating:
Use tf.cast instead.
Epoch 1/50
8000/8000 [==============================] - 2s 267us/step - loss: 0.4877 - accuracy: 0.7949
Epoch 2/50
8000/8000 [==============================] - 1s 149us/step - loss: 0.4328 - accuracy: 0.7960
Epoch 3/50
8000/8000 [==============================] - 1s 155us/step - loss: 0.4285 - accuracy: 0.7960
Epoch 4/50
8000/8000 [==============================] - 1s 152us/step - loss: 0.4245 - accuracy: 0.8039
Epoch 5/50
8000/8000 [==============================] - 1s 148us/step - loss: 0.4221 - accuracy: 0.8226
Epoch 6/50
8000/8000 [==============================] - 1s 148us/step - loss: 0.4199 - accuracy: 0.8266
Epoch 7/50
8000/8000 [==============================] - 1s 171us/step - loss: 0.4189 - accuracy: 0.8289
Epoch 8/50
8000/8000 [==============================] - 1s 152us/step - loss: 0.4174 - accuracy: 0.8305
Epoch 9/50
8000/8000 [==============================] - 1s 146us/step - loss: 0.4166 - accuracy: 0.8317
Epoch 10/50
8000

# Part 3 - Making predictions and evaluating the model


In [17]:
# Predicting the Test set result
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [18]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[ 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

In [19]:
new_prediction 
#the customer doesn't leave the bank !

array([[False]])

In [20]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [21]:
cm

array([[1546,   49],
       [ 269,  136]])

###  Part 4 - Evaluating, Improving and Tuning the ANN

In [22]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 50)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

In [23]:
mean = accuracies.mean()
variance = accuracies.std()

In [29]:
#Improving the ANN
#Dropout Regularization to reduce overfitting if needed

# Tuning the ANN
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)



parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}


In [ ]:
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_